# <center>__MÉTODOS NUMÉRICOS__</center>
## <center>__PROJETO DA UNIDADE 2__</center>

#### <center>__ALUNO:__ Lucas Cordeiro Vieira</center>

<div class="alert alert-block alert-info">
1. INTRODUÇÃO
</div>

Nessa seção você deverá fazer uma introdução sobre o tema de seu projeto, apresentando as características das técnicas trabalhadas durante o trabalho.

PageRank com Decomposições em Auto valores e Auto vetores

Depois que os primeiros computadores pessoais foram criados e começaram a ser comercializados, surgiu a necessidade de ter ferramentas de pesquisa. Como poderia ser possível pesquisar, estudar ou assistir algo que está tão longe de nós? Isso foi possível graças as ferramentas de buscas criadas ao longo dos anos por profissionais da área. Só que para ter o melhor resultado, seria necessário ter um ranking das melhores páginas para retornar ao usuário a busca desejada. Para isso, um método chamado pagerank foi criado pelos fundadores da empresa Google, onde utiliza meios para "rankear" as páginas em que o usuário pesquisa.


<div class="alert alert-block alert-info">
2. DESCRIÇÃO DO PROBLEMA
</div>

Nessa seção você descreverá o problema que será atacado, com o máximo de precisão possível. Tente apresentar as características do problema, bem como as aplicações associadas a ele.

Nosso problema em questão é o método PageRank, criado e utilizado pela Google, um software para classificação de sites. No início, anos 90, os rankings eram feitos baseados em texto, ou seja, quanto mais repetições, mais autoridade teria o site. Só que isso era um problema porque sites em que tinham mais repetições sempre ficavam a cima de outros. Por isso o PageRank foi feito, utilizando mais do que textos simples para "rankear" sites. Além de ser usado como mecanismo de busca pela Google, o método em questão pode ser usado também para identificar se algum usuário é influente ou não em uma determinada página na internet, ou seja, análise de tráfego na rede. Outro cenário de grande utilidade para o método é a detecção de fraudes na indústria farmacêutica, muitos países sofrem com as fraudes médicas, e por isso o algoritmo pagerank pode identificar o problema.



<div class="alert alert-block alert-info">
3. MÉTODOS APLICADOS À SOLUÇÃO
</div>

Métodos utilizados neste trabalho:
* SVD 
Para tentar solucionar o problema de remoção de fundo com PCA robusto, foi utilizado os seguintes métodos: SVD (Decomposição em valores singulares), onde foi feita a fatorização de matrizes, formalmente falando, ela é dita por: M = UΣV*, onde U(m×m, real ou complexa) e V*( n×n, real ou complexa, conjugada transposta de V) são matrizes unitárias e Σ(sigma) é uma matriz retangular diagonal m×n com números reais não-negativos na diagonal.Como funciona o SVD? M é uma matriz onde temos números,reais ou complexos, retangulares, onde esses números escalares passam por uma fatorização utilizando a fórmula: M = UΣV*. Algumas aplicações com este método são: pseudoinversa, aproximação de matrizes, determinação do posto, imagem e núcleo de uma matriz e compressão e imagens. SVD foi utilizado para fazer aproximações das matrizes da imagem, gerando vários vetores onde quanto maior o número de vetores e, dessa forma, conseguindo chegar na imagem original ( diminuindo o tamanho do dado original).


<div class="alert alert-block alert-info">
4. IMPLEMENTAÇÃO
</div>

* Iremos começar importando as bibliotecas necessárias para fazer o trabalho.



In [9]:
import os
import psutil
from time import sleep
from tqdm import tqdm

* A biblioteca psutil é perfeita para vermos o uso de memória.

In [4]:
process = psutil.Process(os.getpid())
t = process.memory_info()

In [5]:
t.vms, t.rss

(68857856, 65486848)

In [6]:
def mem_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / psutil.virtual_memory().total

In [7]:
mem_usage()

0.007734320624855

* Vamos utilizar uma biblioteca TQDM para nos fornecer uma barra de progresso.

In [10]:
s = 0
for i in tqdm(range(10)):
    s += i
    sleep(0.2)
print(s)

100%|█████████████████████████████████████████████| 10/10 [00:02<00:00,  4.83it/s]

45


* Lembrando um pouco sobre o que é SVD...
    * Em álgebra linear, a decomposição em valores singulares ou singular value decomposition é a fatoração de uma matriz real ou complexa, com diversas aplicações importantes em processamento de sinais e estatística.
* Decomposição em valores singulares X Decomposição de valores
    * Relação entre SVD e Decomposição de valores: os vetores singulares à esquerda de A são os autovetores de AAT. Os vetores à direita do singular de A são os autovetores de ATA. Os valores singulares diferentes de zero de A são as raízes quadradas dos autovalores de ATA (e AAT).
    * SVD é uma generalização da decomposição própria. Nem todas as matrizes têm valores próprios, mas TODAS as matrizes têm valores singulares.
    

* Iremos utilizar o conjunto de datos DBpedia que fornece os dados estruturados da wikipedia ( site de informações ).

In [ ]:
import os, numpy as np, pickle
from bz2 import BZ2File
from datetime import datetime
from pprint import pprint
from time import time
from tqdm import tqdm_notebook
from scipy import sparse

from sklearn.decomposition import randomized_svd
from sklearn.externals.joblib import Memory
from urllib.request import urlopen

In [ ]:
PATH = 'data/dbpedia/'
URL_BASE = 'http://downloads.dbpedia.org/3.5.1/en/'
filenames = ["redirects_en.nt.bz2", "page_links_en.nt.bz2"]

for filename in filenames:
    if not os.path.exists(PATH+filename):
        print("Downloading '%s', please wait..." % filename)
        open(PATH+filename, 'wb').write(urlopen(URL_BASE+filename).read())

* Carregando os dados do banco..

In [ ]:
redirects_filename = PATH+filenames[0]
page_links_filename = PATH+filenames[1]

In [ ]:
DBPEDIA_RESOURCE_PREFIX_LEN = len("http://dbpedia.org/resource/")
SLICE = slice(DBPEDIA_RESOURCE_PREFIX_LEN + 1, -1)

* Pegando o o tamanho da banco e fatiando.

In [ ]:
def get_lines(filename): return (line.split() for line in BZ2File(filename))

* Fazendo um loop pelos redirecionamentos e criando um dicionário de origem para o destino final.

In [ ]:
def get_redirect(targ, redirects):
    seen = set()
    while True:
        transitive_targ = targ
        targ = redirects.get(targ)
        if targ is None or targ in seen: break
        seen.add(targ)
    return transitive_targ

In [ ]:
def get_redirects(redirects_filename):
    redirects={}
    lines = get_lines(redirects_filename)
    return {src[SLICE]:get_redirect(targ[SLICE], redirects) 
                for src,_,targ,_ in tqdm_notebook(lines, leave=False)}

In [ ]:
redirects = get_redirects(redirects_filename)

* Vendo o uso da memória até o momento.

In [ ]:
mem_usage()

In [ ]:
def add_item(lst, redirects, index_map, item):
    k = item[SLICE]
    lst.append(index_map.setdefault(redirects.get(k, k), len(index_map)))

In [ ]:
index_map = dict()
lines = get_lines(page_links_filename)
source, destination, data = [],[],[]
for l, split in tqdm_notebook(enumerate(lines), total=limit):
    if l >= limit: break
    add_item(source, redirects, index_map, split[0])
    add_item(destination, redirects, index_map, split[2])
    data.append(1)

In [ ]:
n=len(data); n

In [ ]:
index_map.popitem()

In [ ]:
[i for i,x in enumerate(source) if x == 9991173]

In [ ]:
source[119077649], destination[119077649]

In [ ]:
for page_name, index in index_map.items():
    if index == 9991050:
        print(page_name)

In [ ]:
test_inds = [i for i,x in enumerate(source) if x == 9991050]

In [ ]:
len(test_inds)

In [ ]:
test_inds[:5]

In [ ]:
test_dests = [destination[i] for i in test_inds]

In [ ]:
for page_name, index in index_map.items():
    if index in test_dests:
        print(page_name)

<div class="alert alert-block alert-info">
5. CASOS DE USO
</div>